# 🎨 管理图标（标准结构）

转换、批量处理、更新图标

In [ ]:
import json
import gzip
import base64
from pathlib import Path
from PIL import Image
import io

print("✓ 模块加载成功")

## ⚙️ 配置

In [ ]:
VERSION = "custom"
BASE = Path("..") / "bin" / "version" / VERSION
ICONS = BASE / "quests_icons"

if BASE.exists():
    print(f"✓ 版本: {VERSION}")
else:
    print(f"❌ 版本不存在: {VERSION}")

## 🔧 工具函数

In [ ]:
def img_to_b64(path, fmt='webp', quality=95):
    """图片转Base64"""
    try:
        with Image.open(path) as img:
            if fmt == 'jpeg' and img.mode in ('RGBA', 'LA', 'P'):
                bg = Image.new('RGB', img.size, (255,255,255))
                if img.mode == 'P': img = img.convert('RGBA')
                bg.paste(img, mask=img.split()[-1] if img.mode=='RGBA' else None)
                img = bg
            
            buf = io.BytesIO()
            if fmt == 'webp':
                img.save(buf, 'WebP', quality=quality, method=6)
            elif fmt == 'png':
                img.save(buf, 'PNG', optimize=True)
            else:
                img.save(buf, 'JPEG', quality=quality, optimize=True)
            
            buf.seek(0)
            return base64.b64encode(buf.read()).decode()
    except Exception as e:
        print(f"❌ {path}: {e}")
        return None

def save_icon_file(line, icons_dict):
    """保存图标文件（JSON+GTBL）
    
    参数:
        line: 任务线名称
        icons_dict: {quest_id: base64, ...}
    """
    json_path = ICONS / "QuestIcon" / f"{line}.json"
    gtbl_path = json_path.with_suffix('.gtbl')
    
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(icons_dict, f, ensure_ascii=False, indent=2)
    
    with gzip.open(gtbl_path, 'wb') as f:
        f.write(json.dumps(icons_dict, ensure_ascii=False).encode('utf-8'))
    
    j_size = json_path.stat().st_size
    g_size = gtbl_path.stat().st_size
    print(f"  ✓ {gtbl_path.relative_to(Path('..'))}")
    print(f"    JSON: {j_size:,}B  GTBL: {g_size:,}B  ({(1-g_size/j_size)*100:.1f}% 压缩)")

print("✓ 工具函数就绪")

## 📤 单个图标转换

In [ ]:
IMG_PATH = "path/to/icon.png"
FORMAT = "webp"  # webp, png, jpeg
QUALITY = 95

if Path(IMG_PATH).exists():
    b64 = img_to_b64(IMG_PATH, FORMAT, QUALITY)
    if b64:
        print(f"✓ 转换成功: {len(b64)} 字符")
        print(f"预览: {b64[:100]}...")
        CONVERTED = b64  # 保存供后续使用
else:
    print(f"❌ 文件不存在: {IMG_PATH}")

## 🔄 批量转换文件夹

In [ ]:
SRC_DIR = Path("path/to/icons")
FORMAT = "webp"
QUALITY = 95

if SRC_DIR.exists():
    files = []
    for ext in ['.png', '.jpg', '.jpeg', '.webp', '.bmp']:
        files.extend(SRC_DIR.glob(f"*{ext}"))
    
    if files:
        print(f"找到 {len(files)} 个图标\n")
        BATCH = {}
        
        for f in files:
            name = f.stem
            print(f"{name} ... ", end="")
            b64 = img_to_b64(f, FORMAT, QUALITY)
            if b64:
                BATCH[name] = b64
                print(f"✓ ({len(b64)}字符)")
            else:
                print("❌")
        
        print(f"\n✓ 成功: {len(BATCH)}/{len(files)}")
    else:
        print(f"❌ 无图标文件")
else:
    print(f"❌ 目录不存在")

## 📝 更新任务图标

In [ ]:
# ========== 配置 ==========
QUESTLINE = "WelcomeQuests"   # 任务线名称
QUEST_ID = "welcome_12345"    # 任务ID
NEW_ICON = "your_base64"      # 新图标Base64

# 或使用上面转换的结果
# NEW_ICON = CONVERTED

json_path = ICONS / "QuestIcon" / f"{QUESTLINE}.json"

if json_path.exists():
    with open(json_path, encoding='utf-8') as f:
        icons = json.load(f)
    
    if QUEST_ID in icons:
        old = len(icons[QUEST_ID])
        icons[QUEST_ID] = NEW_ICON
        new = len(NEW_ICON)
        print(f"✓ 更新: {QUEST_ID}")
        print(f"  旧: {old:,} → 新: {new:,} ({new-old:+,})")
    else:
        icons[QUEST_ID] = NEW_ICON
        print(f"✓ 新增: {QUEST_ID}")
    
    save_icon_file(QUESTLINE, icons)
else:
    print(f"❌ 文件不存在: {json_path}")

## 📦 批量更新（使用BATCH结果）

In [ ]:
QUESTLINE = "WelcomeQuests"

if 'BATCH' in globals():
    json_path = ICONS / "QuestIcon" / f"{QUESTLINE}.json"
    
    if json_path.exists():
        with open(json_path, encoding='utf-8') as f:
            icons = json.load(f)
        
        updated, added = 0, 0
        for qid, b64 in BATCH.items():
            if qid in icons:
                updated += 1
            else:
                added += 1
            icons[qid] = b64
        
        print(f"✓ 更新: {updated} | 新增: {added} | 总计: {len(icons)}")
        save_icon_file(QUESTLINE, icons)
    else:
        print(f"❌ 文件不存在")
else:
    print("❌ 请先运行批量转换")

## 📊 统计

In [ ]:
print("="*60)
print(f"📊 {VERSION} 图标统计")
print("="*60)

# 任务线图标
line_dir = ICONS / "QuestLineIcon"
line_count = sum(1 for _ in line_dir.glob("*.gtbl"))
print(f"\n任务线图标: {line_count}")

# 任务图标
quest_dir = ICONS / "QuestIcon"
total_icons = 0
total_json = 0
total_gtbl = 0

for gtbl in quest_dir.glob("*.gtbl"):
    json_file = gtbl.with_suffix('.json')
    if json_file.exists():
        with open(json_file, encoding='utf-8') as f:
            data = json.load(f)
            count = len(data)
            total_icons += count
        
        j_size = json_file.stat().st_size
        g_size = gtbl.stat().st_size
        total_json += j_size
        total_gtbl += g_size
        
        print(f"  {gtbl.stem}: {count} 个图标")

print(f"\n任务图标总计: {total_icons}")
print(f"  JSON: {total_json:,}B")
print(f"  GTBL: {total_gtbl:,}B")
if total_json > 0:
    print(f"  压缩率: {(1-total_gtbl/total_json)*100:.1f}%")

print("="*60)